# El MPJPE (Mean Per Joint Position Error) 

Es una métrica comúnmente utilizada para evaluar la precisión de los modelos de estimación de pose. Calcula la distancia promedio entre las posiciones predichas y las posiciones reales (ground truth) de las articulaciones (joints) en un espacio 3D o 2D.

La distancia euclidiana se calcula con la siguiente formula

![distancia euclidiana](distancia_euclidiana.png)

Donde:
- _p,q_ son dos puntos en el espacio n euclidiano
- _qi, pi_ son vectores euclidianos a partir del origen del espacio
- _n_ es el espacio

In [ ]:
import os
import importlib , mpjpe, pandas as pd # Importar importlib para recargar módulos
from dotenv import load_dotenv

dotenv_path = os.path.join(os.getcwd(), '..', '.env')
importlib.reload(mpjpe)
from mpjpe import MPJPE

# Configuracion
images_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('IMAGES_SUBPATH'))
labels_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('LABELS_SUBPATH'))
evaluator = MPJPE(os.getenv('BASE_PATH'), images_path, labels_path)
thresholds = [0.1, 0.5, 0.9]

# Evaluando una imagen

In [ ]:
image = 'imagen_011_jpg.rf.efc4f6570869d250cc6316a35f6b4b26.jpg'
results = []

for threshold in thresholds:
    results = evaluator.evaluate_image(image, threshold, results)

# Evaluando multiples imagenes

In [2]:
# Obtener la lista de todas las imágenes en la carpeta
image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
image_files_sorted = sorted(image_files)
results = []

for image in image_files_sorted:
    for threshold in thresholds:
        results = evaluator.evaluate_image(image, threshold, results)


image 1/1 /Users/apple/Documents/pose_estimation/dataset/images/imagen_001_jpg.rf.2a634f01a533e3e1518c478c8d4dde0f.jpg: 640x384 1 person, 90.9ms
Speed: 2.9ms preprocess, 90.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs/pose/predict2

image 1/1 /Users/apple/Documents/pose_estimation/dataset/images/imagen_001_jpg.rf.2a634f01a533e3e1518c478c8d4dde0f.jpg: 640x384 1 person, 72.5ms
Speed: 1.0ms preprocess, 72.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs/pose/predict2

image 1/1 /Users/apple/Documents/pose_estimation/dataset/images/imagen_001_jpg.rf.2a634f01a533e3e1518c478c8d4dde0f.jpg: 640x384 1 person, 76.3ms
Speed: 1.1ms preprocess, 76.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs/pose/predict2

image 1/1 /Users/apple/Documents/pose_estimation/dataset/images/imagen_002_jpg.rf.39766b197a04d8d1569b3e546fd07390.jpg: 640x384 1 person, 79.6ms
Speed: 1.1ms prep

# Generando XLS con informacion

In [3]:
# Crear un DataFrame con los resultados
df_results = pd.DataFrame(results)
output_excel_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('YOLO_SUBPATH'), 'precision', 'MPJPE', 'yolov11_mpjpe_results.xlsx')
df_results.to_excel(output_excel_path, index=False, engine='openpyxl')

print(f"Los resultados se han guardado en {output_excel_path}")

Los resultados se han guardado en /Users/apple/Documents/pose_estimation/YOLOv11/precision/MPJPE/yolov11_mpjpe_results.xlsx


# Interpretacion de resultados

Si el MPJPE es alto o bajo se debe considerar que
- Las coordenadas ya se encuentran normalizadas en un valor de 0 y 1
- En terminos de pixeles, si la imagen tiene un tamanho de 256 x 256, el error promedio (5.54%) seria **mpjep x pixeles**, ej. 0.055465 x 256 = 14.19, esto significa que las predicciones estan desviadas por aproximadamente 14.19 pixeles de las reales